In [251]:
import nltk
from nltk.corpus import sentiwordnet as swn
from pycorenlp import StanfordCoreNLP
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
import requests
import sys, os
import numpy as np
import pandas as pd

In [252]:
nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

In [253]:
breakdown = swn.senti_synset('good.n.01')
print(breakdown.pos_score())

0.5


In [254]:
wn.synsets('love')

[Synset('love.n.01'),
 Synset('love.n.02'),
 Synset('beloved.n.01'),
 Synset('love.n.04'),
 Synset('love.n.05'),
 Synset('sexual_love.n.02'),
 Synset('love.v.01'),
 Synset('love.v.02'),
 Synset('love.v.03'),
 Synset('sleep_together.v.01')]

In [255]:
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
op_set = positive_lexicon + negative_lexicon

In [256]:
def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append((pos['word'], pos['pos']))
    return res
    
# def pos_tag(sentence):
#     '''url = "http://localhost:9000"
#     request_params = {"annotators": "pos"}
#     r = requests.post(url, data=sentence, params=request_params, timeout=120)
#     try:
#         results = r.json()['sentences'][0]['tokens']
#         res = []
#         for pos in results:
#             res.append((pos['word'], pos['pos']))
#         return res
#     except Exception as e:
#         print(e)
#         return []
#     '''
#     return nltk.pos_tag(nltk.word_tokenize(sentence))

In [320]:
negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection",
#     "don't",
#     "shouldn't",
#     "wouldn't",
#     "didn't",
#     "though",
#     "although",
#     "wasn't",
#     "isn't",
#     "but"
]

from nltk.corpus import sentiwordnet as swn
import string

def predict_sentiwordnet(opinions, sent):
    endscore = 0
    for opinion in opinions:
        if opinion in positive_lexicon:
            endscore += 1
        elif opinion in negative_lexicon:
            endscore -= 1
        break
        try:
            score = swn.senti_synset(opinion + '.a.1')
            if score.pos_score() > score.neg_score():
                endscore += score.pos_score()
            else:
                endscore += score.neg_score() * (-1)

            words = sent.split(' ')
            word_around = []
            for x in range(0, len(words)):
                if words[x] in string.punctuation:
                    continue
                try:
                    if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                        word_around.append(words[x])
                    elif (words[x-1] == opinion):
                        word_around.append(words[x])
                except:
                    pass
            for neg in negation:
                if neg in word_around:
                    endscore *= (-1)
                    break
        except:
            pass
    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity

def predict_sentiwordnet_lesk(opinions, sent):
    endscore = 0
    for opinion in opinions:
        if opinion in positive_lexicon:
            endscore += 1
        elif opinion in negative_lexicon:
            endscore -= 1
#         break
        try:
            tagged_sentence = pos_tag(sent)
            op_with_tag = ('','')
            for word, tag in tagged_sentence:
                if opinion == word:
                    op_with_tag = (word, tag)
                    break
                    
            pos = ''
            if 'NN' in op_with_tag[1]:
                pos = 'n'
            elif 'JJ' in op_with_tag[1]:
                pos = 'a'
            elif 'VB' in op_with_tag[1]:
                pos = 'v'
            elif 'RB' in op_with_tag[1]:
                pos = 'r'
            else:
                pos = 'a'
            
            score = swn.senti_synset(simple_lesk(sent, opinion, pos).name())
            if score.pos_score() > score.neg_score():
                endscore += score.pos_score()
            else:
                endscore += score.neg_score() * (-1)
                
            words = sent.split(' ')
            word_around = []
            for x in range(0, len(words)):
                if words[x] in string.punctuation:
                    continue
                try:
                    if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                        word_around.append(words[x])
                    elif (words[x-1] == opinion):
                        word_around.append(words[x])
                except:
                    pass
            
            for neg in negation:
                if neg in word_around:
                    endscore *= (-1)
                    break
           
        except Exception as e:
            print(e)
            pass
            
    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity

In [333]:
def calculate(category, is_lesk):
    xtra = ''
    if is_lesk:
        xtra = 'lesk_'
    df = pd.read_csv('Results/Sentiwordnet/hasil_'+ xtra + category +'.csv')
    tp, tn, fp, fn = 0, 0, 0, 0
    p = 'positive'
    n = 'negative'
    for index in range(0, len(df)):
        label = df['label'][index]
        predict = df['predict'][index]
        
        if label == p and predict == p:
            tp += 1
        elif label == n and predict == n:
            tn += 1
        elif label == n and predict == p:
            fp += 1
        elif label == p and predict == n:
            fn += 1
    
    precision = round(tp / (tp + fp), 3)
    recall = round(tp / (tp + fn), 3)
    f1 = round(2 * ((precision * recall) / (precision + recall)), 3)
    accuracy = round((tp+tn)/(tp+tn+fp+fn), 3)
    print(tp, '\t', tn, '\t', fp, '\t', fn, '\t', '\t', precision, '\t', recall, '\t', f1, '\t', '\t', accuracy)
    return tp, tn, fp, fn, precision, recall, f1, accuracy

In [322]:
def preprocessing(sentence):
    return sentence.lower()

In [323]:
import numpy as np

def print_hasil(is_lesk):
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', 'prec', '\t', 'rec', '\t', 'f1', '\t', '\t', 'accuracy')
    precision = []
    recall = []
    f1 = []
    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'PRICES']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, is_lesk)
        precision.append(e)
        recall.append(f)
        f1.append(g)
    #     sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
    #     sf.to_excel("Results/Calculation/"+ tipe +".xlsx")
    print('', '\t', '', '\t', '', '\t', '', '\t', '\t', round(np.mean(precision),2), '\t', round(np.mean(recall),2), '\t', round(np.mean(f1),2), '\t', '\t')

In [324]:
import pandas as pd
def run(category):
    df = pd.read_csv('Results/'+ category +'.csv')
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'predict'])
    for index in range(0, len(df)):
        opinion = [df['opinion'][index]]
        prediction = predict_sentiwordnet(opinion, preprocessing(df['review'][index]))
        sf = sf.append({'id': df['id'][index], 
            'review': df['review'][index],
            'opinion': df['opinion'][index],
            'label': df['label'][index],
            'predict': prediction
           }, ignore_index=True)
    
    sf.to_csv("Results/Sentiwordnet/hasil_"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/hasil_"+ category +".xlsx")

In [325]:
import pandas as pd
def run_lesk(category):
    df = pd.read_csv('Results/'+ category +'.csv')
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'predict'])
    for index in range(0, len(df)):
        opinion = [df['opinion'][index]]
        prediction = predict_sentiwordnet_lesk(opinion, preprocessing(df['review'][index]))
        sf = sf.append({'id': df['id'][index], 
            'review': df['review'][index],
            'opinion': df['opinion'][index],
            'label': df['label'][index],
            'predict': prediction
           }, ignore_index=True)
    
    sf.to_csv("Results/Sentiwordnet/hasil_lesk_"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/hasil_lesk_"+ category +".xlsx")

In [326]:
run('FOOD')
run('AMBIENCE')
run('SERVICE')
run('PRICES')

In [327]:
run_lesk('FOOD')
run_lesk('AMBIENCE')
run_lesk('SERVICE')
run_lesk('PRICES')

'float' object has no attribute 'endswith'
'NoneType' object has no attribute 'name'
'float' object has no attribute 'endswith'
'float' object has no attribute 'endswith'
'float' object has no attribute 'endswith'
'NoneType' object has no attribute 'name'
'NoneType' object has no attribute 'name'
'NoneType' object has no attribute 'name'
'NoneType' object has no attribute 'name'
'NoneType' object has no attribute 'name'


In [332]:
print_hasil(False)

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
76 	 10 	 10 	 9 	 	 0.88 	 0.89 	 0.88 	 	 0.82
107 	 24 	 12 	 28 	 	 0.9 	 0.79 	 0.84 	 	 0.77
63 	 25 	 16 	 15 	 	 0.8 	 0.81 	 0.8 	 	 0.74
37 	 25 	 11 	 6 	 	 0.77 	 0.86 	 0.81 	 	 0.78
 	  	  	  	 	 0.84 	 0.84 	 0.83 	 	


In [331]:
print_hasil(True)

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
78 	 10 	 10 	 7 	 	 0.89 	 0.92 	 0.9 	 	 0.84
109 	 25 	 11 	 26 	 	 0.91 	 0.81 	 0.86 	 	 0.78
66 	 24 	 17 	 12 	 	 0.8 	 0.85 	 0.82 	 	 0.76
37 	 24 	 12 	 6 	 	 0.76 	 0.86 	 0.81 	 	 0.77
 	  	  	  	 	 0.84 	 0.86 	 0.85 	 	
